In [1]:
using VCF

┌ Info: Precompiling VCF [8f8d80d9-e796-45a3-8ef2-c0461a3110a4]
└ @ Base loading.jl:1278
┌ Warning: Package VCF does not have CodecZlib in its dependencies:
│ - If you have VCF checked out for development and have
│   added CodecZlib as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with VCF
└ Loading CodecZlib into VCF from project dependency, future warnings for VCF are suppressed.


## read_vcf

In [ ]:
read_vcf("example.vcf.gz")